# 환경설정

In [19]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [20]:
from langchain_neo4j import Neo4jGraph

# Neo4jGraph 인스턴스 생성
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

# llm_client 
    - LLM 모델을 커스터마이징

In [52]:
from graphiti_core import Graphiti
from datetime import datetime
from graphiti_core.llm_client.openai_client import OpenAIClient
from graphiti_core.llm_client.config import LLMConfig


# 커스텀 LLM 설정 (OpenAI)
custom_llm = OpenAIClient(
    config=LLMConfig(
        api_key=os.getenv("OPENAI_API_KEY"),
        model="gpt-4o",             # 메인 모델
        small_model="gpt-4o-mini"   # 작은 작업용 모델
    )
)

# Graphiti 인스턴스 생성
graphiti = Graphiti(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),     
    password=os.getenv("NEO4J_PASSWORD"),
    llm_client=custom_llm  # 커스텀 설정
)

# 설정 확인
print(f"사용 중인 모델: {graphiti.llm_client.config.model}")
print(f"작은 모델: {graphiti.llm_client.config.small_model}")

사용 중인 모델: gpt-4o
작은 모델: gpt-4o-mini


# graphiti 초기화
- Neo4j 데이터베이스에 필요한 인덱스와 제약조건을 생성하는 초기화 함수 

## 실제 생성하는 것들
- 인덱스, 벡터 인덱스 (Index, Vector Index)
- 풀텍스트 인덱스 (Fulltext Index)
- 제약조건 (Constraints)

## 실제로 실행되는 Cypher 쿼리들
1. 엔티티 (Entity) 관련
    - CREATE VECTOR INDEX entity_embeddings FOR (n:Entity) ON n.embedding
    - CREATE FULLTEXT INDEX entity_fulltext FOR (n:Entity) ON [n.name, n.summary]
    - CREATE CONSTRAINT entity_uuid FOR (n:Entity) REQUIRE n.uuid IS UNIQUE

2. 엣지/관계 (Edge) 관련
    - CREATE VECTOR INDEX edge_embeddings FOR ()-[r:RELATES_TO]-() ON r.embedding
    - CREATE FULLTEXT INDEX edge_fulltext FOR ()-[r:RELATES_TO]-() ON [r.fact]
    - CREATE CONSTRAINT edge_uuid FOR ()-[r:RELATES_TO]-() REQUIRE r.uuid IS UNIQUE

3. 에피소드 (Episode) 관련
    - CREATE CONSTRAINT episode_uuid FOR (n:Episode) REQUIRE n.uuid IS UNIQUE
    - CREATE INDEX episode_created FOR (n:Episode) ON n.created_at

4. 커뮤니티 (Community) 관련 (선택적)
    - CREATE CONSTRAINT community_uuid FOR (n:Community) REQUIRE n.uuid IS UNIQUE   

5. 시간 관련 인덱스
    - CREATE INDEX edge_valid_from FOR ()-[r:RELATES_TO]-() ON r.valid_from
    - CREATE INDEX edge_valid_to FOR ()-[r:RELATES_TO]-() ON r.valid_to

6. 그룹 ID 인덱스 (멀티테넌시)
    - CREATE INDEX entity_group_ids FOR (n:Entity) ON n.group_ids
    - CREATE INDEX edge_group_ids FOR ()-[r:RELATES_TO]-() ON r.group_ids

In [ ]:

from graphiti_core import Graphiti
from datetime import datetime

# 최초 한 번만 실행 (테이블 생성 같은 것)
await graphiti.build_indices_and_constraints()
        
print("✅ 초기화 완료!")

✅ 초기화 완료!


# 인덱스 확인

In [23]:
# 벡터 인덱스 확인
check_vector_index_query = """
SHOW INDEXES
"""
vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    # 벡터 인덱스 정보 출력
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")    
    print(f"Property Key: {index['properties']}")
    print("-" * 40)

Index Name: community_group_id
Type: RANGE
Property Key: ['group_id']
----------------------------------------
Index Name: community_name
Type: FULLTEXT
Property Key: ['name', 'group_id']
----------------------------------------
Index Name: community_uuid
Type: RANGE
Property Key: ['uuid']
----------------------------------------
Index Name: created_at_edge_index
Type: RANGE
Property Key: ['created_at']
----------------------------------------
Index Name: created_at_entity_index
Type: RANGE
Property Key: ['created_at']
----------------------------------------
Index Name: created_at_episodic_index
Type: RANGE
Property Key: ['created_at']
----------------------------------------
Index Name: edge_name_and_fact
Type: FULLTEXT
Property Key: ['name', 'fact', 'group_id']
----------------------------------------
Index Name: entity_group_id
Type: RANGE
Property Key: ['group_id']
----------------------------------------
Index Name: entity_uuid
Type: RANGE
Property Key: ['uuid']
-----------------

# 데이터저장 (add_episode)
    - 데이터 저장 시 내부적으로 LLM을 호출하여 노드 정의 및 관계 정의 생성한다.

# 데이터 저장시 내부적으로
1. 텍스트 전처리                    (~10ms)
2. LLM 엔티티 추출 ⭐               (~500ms)  ← 가장 느림
3. 임베딩 생성 (병렬)               (~200ms)
4. 기존 엔티티 검색 (벡터 유사도)    (~100ms)
5. 엔티티 해상도 결정               (~50ms)
6. 기존 관계 조회                   (~30ms)
7. 충돌 감지 (LLM) ⭐               (~500ms)  ← 두 번째로 느림
8. 관계 무효화                      (~20ms)
9. 새 관계 생성                     (~30ms)
10. 인덱스 업데이트                 (~50ms)
11. 커뮤니티 재계산                 (~100ms)

-> LLM 호출: 2회

# 복잡한 이유?
데이터 정합성을 보장해야 함 (엔티티 중복 확인 필요, 충돌 감지 필요, 시간 무효화 처리 필요, llm으로 의미 이해 필요)

# 저장 시간 소요
- 짧은 문장(10~20 단어 이내): 10-20초
- 중간 문장 (50단어): 20-40초
- 긴 문단 (200단어): 40-90초

In [ ]:

await graphiti.add_episode(
    name="first_memory",               # 에피소드 식별자
    episode_body="철수는 개발자다",    # 저장할 내용
    source_description="사용자 입력",  # 데이터 출처 설명
    reference_time=datetime.now()      # 현재 시간
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: entity_edges)} {position: line: 17, column: 7, offset: 473} for query: '\n                                    MATCH (e:Episodic)\n                                    WHERE e.valid_at <= $reference_time\n                                    \nAND e.group_id IN $group_ids\nAND e.source = $source\n        RETURN\n        \n    e.uuid AS uuid,\n    e.name AS name,\n    e.group_id AS group_id,\n    e.created_at AS created_at,\n    e.source AS source,\n    e.source_description AS source_description,\n    e.content AS content,\n    e.valid_at AS valid_at,\n    

AddEpisodeResults(episode=EpisodicNode(uuid='80a79ea9-20b7-48f8-9cf8-c0197c0314bf', name='first_memory', group_id='', labels=[], created_at=datetime.datetime(2025, 12, 2, 3, 57, 15, 642962, tzinfo=datetime.timezone.utc), source=<EpisodeType.message: 'message'>, source_description='사용자 입력', content='철수는 개발자다', valid_at=datetime.datetime(2025, 12, 2, 4, 57, 15, 642956), entity_edges=['bb55a0c8-e209-4b79-9db0-d079082e98b1']), episodic_edges=[EpisodicEdge(uuid='d4cc6f00-8cd4-4d0a-b718-da7fac424977', group_id='', source_node_uuid='80a79ea9-20b7-48f8-9cf8-c0197c0314bf', target_node_uuid='8100d343-71ac-4350-a206-6d8cbf2917e2', created_at=datetime.datetime(2025, 12, 2, 3, 57, 15, 642962, tzinfo=datetime.timezone.utc)), EpisodicEdge(uuid='1bf607a8-3f52-45c2-805c-b731bd39ec97', group_id='', source_node_uuid='80a79ea9-20b7-48f8-9cf8-c0197c0314bf', target_node_uuid='a9104de8-6cde-4be4-bc3e-0de0aa6fc1e6', created_at=datetime.datetime(2025, 12, 2, 3, 57, 15, 642962, tzinfo=datetime.timezone.utc)), E

# 데이터 검색 (search)
    - 검색할때는 LLM 사용안함

## 내부에서 자동으로
- ✅ 풀텍스트 인덱스 검색
- ✅ 벡터 검색 (의미)
- ✅ BM25 검색 (키워드)
- ✅ 그래프 순회 (관계)
- ✅ 점수 결합 (40% + 30% + 30%) / 기본비율 
- 저장(add)와 비교하면 훨씬 빠르다.

## 비율 커스텀 (SearchConfig로 세밀하게 조정 가능)
"""
# 
config = SearchConfig(
    limit=10,                   # 최대 결과 수
    
    # 엣지(관계) 검색 설정
    edges=EdgeConfig(
        semantic_weight=0.5,    # 의미 검색 50%
        fulltext_weight=0.3,    # 키워드 검색 30%
        graph_weight=0.2        # 그래프 순회 20%
    ),
    
    # 노드(엔티티) 검색 설정
    nodes=NodeConfig(
        semantic_weight=0.6,    # 의미 검색 60%
        fulltext_weight=0.4,    # 키워드 검색 40%
        # 노드는 그래프 순회 없음
    )
)

# 커스텀 설정으로 검색
results = await graphiti._search(
    query="우주 영화 추천",
    config=config
)

results = await graphiti._search("검색어", config=config)
"""

## 상황별로도 커스텀 가능
- 위처럼 여러개의 config를 설정하고 상황에 때라 _search 옵션으로 config을 바꿔넣어주면된다.

In [31]:
# 검색
results = await graphiti.search(
    query="개발자",
    num_results=5
)

# 결과 출력
print(f"검색 결과: {len(results)}개\n")
for i, result in enumerate(results, 1):
    print(f"{i}. {result}")

print("=== 구조적으로 출력 ===\n")
for result in results:
    # 엣지(관계)인 경우 - 객체 속성으로 접근
    if hasattr(result, 'fact'):
        print(f"📌 {result.fact}")
    # 노드(엔티티)인 경우  
    elif hasattr(result, 'name'):
        print(f"👤 {result.name}")

# fact만 추출
print("\n=== 사실들만 추출 ===")
results = await graphiti.search("철수", num_results=10)

# 객체 속성으로 접근하도록 수정
facts = [r.fact for r in results if hasattr(r, 'fact')]

for i, fact in enumerate(facts, 1):
    print(f"{i}. {fact}")

검색 결과: 3개

1. uuid='88b04f06-c8f4-4173-9795-2fb86d17b924' group_id='user_chulsoo' source_node_uuid='ef82a93c-41b8-4ed5-90b5-b0fa13a979b2' target_node_uuid='9326cfb0-e12f-47cb-b296-372151234b0c' created_at=datetime.datetime(2025, 12, 2, 4, 26, 6, 347393, tzinfo=<UTC>) name='IS_A' fact='철수는 개발자다' fact_embedding=None episodes=['8ff91563-5730-44c6-ab1c-f2310e601c22'] expired_at=None valid_at=datetime.datetime(2025, 12, 2, 5, 25, 59, 149753, tzinfo=<UTC>) invalid_at=None attributes={}
2. uuid='bb55a0c8-e209-4b79-9db0-d079082e98b1' group_id='' source_node_uuid='a9104de8-6cde-4be4-bc3e-0de0aa6fc1e6' target_node_uuid='f52031a9-655f-4ad9-bcc7-00d324c30672' created_at=datetime.datetime(2025, 12, 2, 3, 57, 27, 920617, tzinfo=<UTC>) name='IS_A' fact='철수는 개발자다' fact_embedding=None episodes=['80a79ea9-20b7-48f8-9cf8-c0197c0314bf', '34117602-4ea7-4dad-a21d-cc90f6060bd3'] expired_at=datetime.datetime(2025, 12, 2, 4, 5, 40, 995849, tzinfo=<UTC>) valid_at=datetime.datetime(2025, 12, 2, 4, 57, 15, 642956

# 그룹 (group_id)

## 그룹별 데이터 저장

In [37]:
# 철수 정보
await graphiti.add_episode(
    name="chulsoo_info",
    episode_body="철수는 개발자다",
    source_description="사용자 입력",
    reference_time=datetime.now(),
    group_id="user_chulsoo"
)

# 영희 정보
await graphiti.add_episode(
    name="younghee_info",
    episode_body="영희는 디자이너다",
    source_description="사용자 입력",
    reference_time=datetime.now(),
    group_id="user_younghee"
)

print("✅ 2명의 데이터 저장 완료!")

✅ 2명의 데이터 저장 완료!


## 그룹별 검색

In [41]:
# 철수 정보만 검색
chulsoo_results = await graphiti.search(
    query="직업",
    group_ids=["user_chulsoo"]
)

print("=== 철수 정보 ===")
for r in chulsoo_results:
    if hasattr(r, 'fact'):
        print(f"  {r.fact}")

# 영희 정보만 검색
younghee_results = await graphiti.search(
    query="직업",
    group_ids=["user_younghee"]
)

print("\n=== 영희 정보 ===")
for r in younghee_results:
    if hasattr(r, 'fact'):
        print(f"  {r.fact}")

=== 철수 정보 ===
  철수는 개발자다

=== 영희 정보 ===


# JSON으로 저장

In [42]:
import json

# 구조화된 데이터
person_data = {
    "name": "홍길동",
    "age": 30,
    "job": "Python 개발자",
    "skills": ["Python", "FastAPI", "Docker"],
    "location": "서울"
}

# JSON으로 저장
await graphiti.add_episode(
    name="person_profile",
    episode_body=json.dumps(person_data, ensure_ascii=False),
    source_description="사용자 입력",
    reference_time=datetime.now(),
    group_id="user_chulsoo"
)

print("✅ JSON 데이터 저장 완료!")
print(f"\n저장된 데이터:\n{json.dumps(person_data, indent=2, ensure_ascii=False)}")

✅ JSON 데이터 저장 완료!

저장된 데이터:
{
  "name": "홍길동",
  "age": 30,
  "job": "Python 개발자",
  "skills": [
    "Python",
    "FastAPI",
    "Docker"
  ],
  "location": "서울"
}


# 텍스트 vs JSON 비교
    - 실행x, 단순 비교만 하기위함

In [ ]:
# 방법 1: 텍스트
await graphiti.add_episode(
    name="text_data",
    episode_body="철수는 30살이고 개발자다. 파이썬을 잘한다.",
    source_description="사용자 입력",
    reference_time=datetime.now(),
    group_id="user_chulsoo"
)

# 방법 2: JSON
await graphiti.add_episode(
    name="json_data",
    episode_body=json.dumps({
        "name": "철수",
        "age": 30,
        "job": "개발자",
        "skill": "Python"
    }, ensure_ascii=False),
    source_description="사용자 입력",
    reference_time=datetime.now(),
    group_id="user_chulsoo"
)

# 에러 처리

In [ ]:
async def safe_save(user_id, message):
    """에러 처리가 있는 저장"""
    try:
        await graphiti.add_episode(
            name=f"chat_{user_id}",
            episode_body=message,
            reference_time=datetime.now(),
            group_id=user_id
        )
        print(f"✅ 저장 성공: {message}")
        return True
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return False

async def safe_search(user_id, query):
    """에러 처리가 있는 검색"""
    try:
        results = await graphiti.search(
            query=query,
            num_results=5,
            group_ids=[user_id]
        )
        print(f"✅ 검색 성공: {len(results)}개 발견")
        return results
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return []

print("✅ 안전한 함수들 준비 완료!")

 # add_episode_bulk 
 - 여러 에피소드를 한 번에 저장 (대량 데이터)

In [ ]:
# 여러 개 한 번에 저장
# add_episode를 100번 호출하면 느림
# add_episode_bulk로 한 번에 보내면 빠름

from graphiti_core.models import EpisodeInput
from datetime import datetime

# 대량 데이터 준비
episodes = []

# 고객 100명의 프로필 한 번에 저장
for i in range(100):
    episode = EpisodeInput(
        name=f"customer_{i}_profile",
        content=f"고객 {i}번은 VIP 등급이며 서울에 거주합니다.",
        source_description="고객 프로필",
        reference_time=datetime.now(),
        group_id=f"customer_{i}"
    )
    episodes.append(episode)

# 한 번에 저장!
await graphiti.add_episode_bulk(episodes)

# 100번 개별 저장 -> 약 50초
# 한 번에 대량 저장 -> 약 약 5초
print(f"✅ {len(episodes)}개 에피소드 저장 완료!")

# add_triplet
- 용도: 직접 그래프 구조 정의 (고급 사용자용)

In [ ]:
# 예: 회사 조직도 데이터가 이미 있음
org_data = [
    {"manager": "김팀장", "relation": "manages", "member": "이대리"},
    {"manager": "김팀장", "relation": "manages", "member": "박사원"},
    {"manager": "이대리", "relation": "manages", "member": "최인턴"},
]

# 직접 그래프로 저장 (빠르고 정확)
for item in org_data:
    await graphiti.add_triplet(
        subject=item["manager"],
        predicate=item["relation"],
        object=item["member"],
        reference_time=datetime.now()
    )

print("✅ 조직도 저장 완료!")

# add_episode vs add_triplet 차이
    - add_episode: LLM이 자동으로 정의하고 생성 (철수)-[근무]->(카카오), (철수)-[직업]->(개발자)
    - add_triplet 직접 정의, LLM 안 씀

** 언제사용?**
- 이미 구조화된 데이터가 있을 때
- 정확한 그래프 구조가 필요할 때

In [ ]:
# add_episode (LLM이 자동 분석)
await graphiti.add_episode(
    name="job_info",
    episode_body="철수는 카카오에서 개발자로 일한다",  # 자연어
    reference_time=datetime.now()
)
# → LLM이 자동으로: (철수)-[근무]->(카카오), (철수)-[직업]->(개발자)


# add_triplet (직접 정의, LLM 안 씀)
await graphiti.add_triplet(
    subject="철수",
    predicate="근무",
    object="카카오"
)
# → 정확히 이 관계만 저장

# build_communities
    - 용도: 관련된 노드들을 자동으로 그룹화

**커뮤니티란?**
- 서로 연결된 노드들의 그룹
- 예: "개발팀", "마케팅팀", "서울 지역 고객들"

** 언제사용?**
- 데이터를 많이 추가한 후
- 주기적으로 (예: 매일 밤)
- 검색 품질을 높이고 싶을 때

In [ ]:
# 1. 먼저 데이터 저장
await graphiti.add_episode(
    episode_body="철수는 파이썬 개발자다",
    ...
)
await graphiti.add_episode(
    episode_body="영희는 자바 개발자다",
    ...
)
await graphiti.add_episode(
    episode_body="민수는 디자이너다",
    ...
)

# 2. 커뮤니티 생성
await graphiti.build_communities()

# 3. 이제 검색할 때 커뮤니티 정보도 활용됨
results = await graphiti.search("개발자")
# → "개발자 커뮤니티" 전체를 찾아줌

# get_nodes_and_edges_by_episode
    - 용도: 특정 에피소드가 어떻게 그래프로 변환됐는지 확인

**왜 필요?**
    - 디버깅: "내 데이터가 제대로 저장됐나?"
    - 확인: "LLM이 어떻게 해석했나?"

In [ ]:
# 1. 에피소드 저장
await graphiti.add_episode(
    name="철수_프로필",
    episode_body="철수는 서울에 사는 30살 파이썬 개발자다",
    reference_time=datetime.now()
)

# 2. 어떻게 저장됐는지 확인
nodes, edges = await graphiti.get_nodes_and_edges_by_episode(
    episode_name="철수_프로필"
)

print("=== 생성된 노드 ===")
for node in nodes:
    print(f"  - {node.name} (타입: {node.type})")

print("\n=== 생성된 관계 ===")
for edge in edges:
    print(f"  - {edge.source} --[{edge.relation}]-> {edge.target}")

"""
**출력 예시:**

=== 생성된 노드 ===
  - 철수 (타입: Person)
  - 서울 (타입: Location)
  - 파이썬 (타입: ProgrammingLanguage)
  - 개발자 (타입: Occupation)

=== 생성된 관계 ===
  - 철수 --[거주]-> 서울
  - 철수 --[나이]-> 30
  - 철수 --[직업]-> 개발자
  - 철수 --[사용언어]-> 파이썬
"""

# close (종료)
    - 프로그램 종료 전
    - 리소스 정리

In [ ]:
async def main():
    graphiti = Graphiti("bolt://localhost:7687", "neo4j", "password")
    
    try:
        # 작업 수행
        await graphiti.add_episode(...)
        results = await graphiti.search(...)
        
    finally:
        # 항상 종료
        await graphiti.close()
        print("✅ 연결 종료")

# 또는 context manager 사용
async with Graphiti(...) as graphiti:
    await graphiti.add_episode(...)
    # 자동으로 close됨

# clients / embedder / cross_encoder
    - 용도: 내부 설정 확인 및 커스터마이징

In [ ]:
# 현재 설정 확인
print(f"LLM 클라이언트: {graphiti.llm_client}")
print(f"임베더: {graphiti.embedder}")
print(f"크로스 인코더: {graphiti.cross_encoder}")

# driver
    - 직접 쿼리 실행

In [ ]:
driver = graphiti.driver
result = await driver.execute_query("MATCH (n) RETURN count(n)")

## 팁
    - 대량 데이터는 bulk 사용
    - 구조화된 데이터는 triplet 사용
    - 주기적으로 커뮤니티 재생성

## 함수요약표
| 함수 | 빈도 | 용도 |
|------|------|------|
| build_indices_and_constraints | 최초 1회 | 초기 설정 | 
| add_episode | 매우 자주 | 데이터 저장 (자연어) |
| search | 매우 자주 | 데이터 검색 |
| add_episode_bulk | 가끔 | 대량 데이터 저장 |
| add_triplet | 가끔 | 직접 그래프 생성 |
| build_communities | 주기적 | 커뮤니티 생성 |
| get_nodes_and_edges_by_episode | 디버깅 | 저장 결과 확인 |
| close | 종료 시 | 연결 종료 |
